<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=40cee3d83286f135e5fa063793e06121387922969de7d988115bb1417b29f142
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 15:13:02
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -19.16 K (-0.13%)
Current PnL: -23.53 L (-15.42%)
CY Booked + Current PnL: -9.64 L (-6.31%)
-------------------
Total profit:  1.28 L
Total loss:  -24.80 L
-------------------
Total Booked + Current PnL: 17.34 L (13.81%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.94 L (62.51%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.17,-14.08,16.49,0.09,14279.0,-14186.0,86591.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.76,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.22,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.04,6.52,12.84,20.19,23842.0,11364.0,185684.0,-16.49,49.0,X-MC,7.57,68.0,0.48,1.31,22.62,X40,ATH,INSURANCE
43,ITC,452.00,-0.64,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,40.0,X-LC,0.84,5.0,-0.16,1.38,3.41,X40,NTT,FMCG
52,MEDANTA,1486.00,-0.04,4.37,18.88,24.08,25016.0,5550.0,132500.0,-4.04,48.0,X-SC,4.92,89.0,0.22,0.93,24.85,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,4.07,-49.29,233.36,69.05,157467.0,-65588.0,67478.0,-58.98,38.0,H-SC,23.55,140.0,-0.42,0.47,4.07,SR,ATH,IT
56,QUESS,424.00,3.67,-26.52,94.41,42.84,45094.0,-17242.0,47764.0,-52.07,44.0,X-SC,37.31,83.0,-0.38,0.34,3.93,XY24,NTT,MISC
61,REPCOHOME,880.00,3.56,-9.90,110.68,89.82,285708.0,-28366.0,258139.0,-55.29,52.0,H-SC,10.48,160.0,-0.10,1.81,36.57,XY24,NTT,FINANCE
38,INDIAMART,4810.62,2.32,1.46,103.75,106.72,129825.0,1797.0,125133.0,-49.68,42.0,H-SC,3.76,139.0,0.01,0.88,25.18,AR,ATH,MISC
14,CAMPUS,393.00,2.29,-7.02,43.35,33.28,67622.0,-11784.0,155991.0,-25.09,51.0,M-SC,2.98,222.0,-0.17,1.10,20.90,XY24,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HONAUT,58357.33,-3.60,-14.14,66.93,43.32,93595.0,-23032.0,139840.0,-26.21,43.0,X-SC,2.01,90.0,-0.25,0.98,7.37,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-2.30,-21.42,108.91,64.15,203629.0,-50981.0,186970.0,-54.13,54.0,X-SC,1.63,82.0,-0.25,1.31,4.88,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-1.85,-63.70,175.69,0.07,87478.0,-87386.0,49791.0,1724.49,54.0,L-SC,1.51,268.0,-1.00,0.35,23.83,OX40N,NTT,JEWELLERY
8,AWL,485.00,-1.80,-13.73,77.92,53.50,202851.0,-41418.0,260333.0,-58.27,51.0,X-MC,4.24,58.0,-0.20,1.83,15.63,XY24,NTT,FMCG
22,DIXON,18931.72,-1.80,-7.49,31.42,21.58,40734.0,-10494.0,129645.0,-53.35,28.0,X-MC,6.89,56.0,-0.26,0.91,14.78,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.76,0.64,71.16,72.27,121356.0,1091.0,170539.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.2,6.86,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.88,-4.45,117.60,107.91,165902.0,-6571.0,141073.0,-22.17,36.0,M-SC,10.44,216.0,-0.04,0.99,0.88,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.22,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.17,-14.08,16.49,0.09,14279.0,-14186.0,86591.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.76,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.54,-23.00,47.36,13.47,98346.0,-62010.0,207657.0,-68.31,25.0,H-SC,1.38,158.0,-0.63,1.46,5.59,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.35,-23.47,30.66,-0.01,29987.0,-30001.0,97804.0,19921.74,55.0,M-SC,1.67,233.0,-1.00,0.69,10.75,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.76,0.64,71.16,72.27,121356.0,1091.0,170539.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.20,6.86,XR,NTT,IT
38,INDIAMART,4810.62,2.32,1.46,103.75,106.72,129825.0,1797.0,125133.0,-49.68,42.0,H-SC,3.76,139.0,0.01,0.88,25.18,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,2.32,1.46,103.75,106.72,129825.0,1797.0,125133.0,-49.68,42.0,H-SC,3.76,139.0,0.01,0.88,25.18,AR,ATH,MISC
85,WIPRO,420.00,-0.76,0.64,71.16,72.27,121356.0,1091.0,170539.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.20,6.86,XR,NTT,IT
25,FINCABLES,1641.55,0.88,-4.45,117.60,107.91,165902.0,-6571.0,141073.0,-22.17,36.0,M-SC,10.44,216.0,-0.04,0.99,0.88,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.24,-2.05,38.46,35.63,78953.0,-4287.0,205287.0,-15.85,35.0,H-MC,4.38,119.0,-0.05,1.44,14.29,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,1.12,-4.12,89.74,81.92,98776.0,-4729.0,110069.0,-43.90,36.0,M-SC,4.58,236.0,-0.05,0.77,30.30,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.09,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-0.58,-9.89,27.87,15.23,50313.0,-19818.0,180529.0,-22.98,23.0,X-MC,6.66,66.0,-0.39,1.27,19.17,X40N,NTT,REALTY
60,RELAXO,1176.00,-0.47,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,26.0,X-SC,3.97,91.0,-0.48,0.53,0.91,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.10,-35.06,108.21,35.20,90416.0,-45114.0,83556.0,9.60,26.0,X-SC,12.86,92.0,-0.50,0.59,0.63,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.80,-7.49,31.42,21.58,40734.0,-10494.0,129645.0,-53.35,28.0,X-MC,6.89,56.0,-0.26,0.91,14.78,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.09,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.64,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,40.0,X-LC,0.84,5.0,-0.16,1.38,3.41,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.83,1.53,29.00,30.97,52036.0,2695.0,179433.0,-22.41,50.0,X-MC,1.58,75.0,0.05,1.26,14.28,X40,ATH,INSURANCE
3,ACC,3906.0,-2.30,-21.42,108.91,64.15,203629.0,-50981.0,186970.0,-54.13,54.0,X-SC,1.63,82.0,-0.25,1.31,4.88,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.61,-14.47,46.75,25.51,74410.0,-26930.0,159165.0,-2.24,55.0,H-LC,1.88,49.0,-0.36,1.12,17.29,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.76,-14.15,72.92,48.45,125520.0,-28372.0,172134.0,-20.30,28.0,X-MC,6.84,64.0,-0.23,1.21,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.89,-5.91,33.99,26.07,52365.0,-9680.0,154060.0,-30.68,33.0,X-MC,9.05,55.0,-0.18,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-1.09,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.10,-35.06,108.21,35.20,90416.0,-45114.0,83556.0,9.60,26.0,X-SC,12.86,92.0,-0.50,0.59,0.63,X40,NTT,FOOTWEAR
55,PGHH,17907.65,-0.06,-4.18,39.60,33.76,76198.0,-8400.0,192420.0,-32.38,34.0,X-MC,3.35,57.0,-0.11,1.35,0.80,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,3.67,-26.52,94.41,42.84,45094.0,-17242.0,47764.0,-52.07,44.0,X-SC,37.31,83.0,-0.38,0.34,3.93,XY24,NTT,MISC
60,RELAXO,1176.00,-0.47,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,26.0,X-SC,3.97,91.0,-0.48,0.53,0.91,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.10,-35.06,108.21,35.20,90416.0,-45114.0,83556.0,9.60,26.0,X-SC,12.86,92.0,-0.50,0.59,0.63,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.80,-7.49,31.42,21.58,40734.0,-10494.0,129645.0,-53.35,28.0,X-MC,6.89,56.0,-0.26,0.91,14.78,X40N,ATH,IT
52,MEDANTA,1486.00,-0.04,4.37,18.88,24.08,25016.0,5550.0,132500.0,-4.04,48.0,X-SC,4.92,89.0,0.22,0.93,24.85,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.62,-15.18,40.63,19.28,119221.0,-52526.0,293430.0,-26.21,58.0,X-LC,7.53,1.0,-0.44,2.06,8.48,X40,ATH,IT
41,INFY,2275.00,-0.97,4.36,48.40,54.87,158040.0,13641.0,326529.0,-18.88,56.0,X-LC,3.01,2.0,0.09,2.30,13.00,X40,BTT,IT
76,TMPV,600.00,-1.09,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.62,-9.33,49.64,35.67,142152.0,-29476.0,286366.0,-20.14,42.0,X-LC,5.95,4.0,-0.21,2.01,3.23,X40N,ATH,FMCG
43,ITC,452.00,-0.64,-2.04,12.73,10.44,24959.0,-4073.0,196065.0,-41.56,40.0,X-LC,0.84,5.0,-0.16,1.38,3.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.61,-12.70,108.11,81.67,88535.0,-11917.0,81893.0,7361.11,52.0,L-SC,18.77,271.0,-0.13,0.58,58.59,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.76,-13.17,60.70,39.54,53319.0,-13318.0,87841.0,-32.10,64.0,M-SC,6.70,220.0,-0.25,0.62,23.86,AR,ATH,AUTO
51,MASFIN,398.61,-0.66,-5.83,29.61,22.05,27320.0,-5715.0,92265.0,-18.70,48.0,H-SC,7.30,164.0,-0.21,0.65,34.11,XR,ATH,FINANCE
70,SURYODAY,216.00,0.14,-19.72,53.26,23.03,76566.0,-35312.0,143759.0,55.70,48.0,H-SC,10.61,167.0,-0.46,1.01,42.64,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.22,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.22,-9.14,10.08,0.02,15988.0,-15951.0,158608.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.11,37.96,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.10,0.54,20.30,20.95,51384.0,1355.0,253123.0,-0.49,76.0,X-LC,13.37,31.0,0.03,1.78,36.37,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.52,-37.95,114.18,32.89,54696.0,-29304.0,47903.0,-691.63,60.0,L-MC,5.58,259.0,-0.54,0.34,31.92,XR,NTT,BANKS
79,UNITDSPR,1644.00,0.11,5.25,14.74,20.77,36326.0,12298.0,246442.0,-3.83,60.0,X-MC,3.57,62.0,0.34,1.73,12.53,X40N,NTT,BREWERIES
12,BERGEPAINT,680.00,-1.60,1.55,19.29,21.14,44535.0,3531.0,230870.0,-5.67,57.0,X-MC,3.83,74.0,0.08,1.62,30.13,XY24,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,44.63
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.70
MC    29.24
LC    25.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      21.20
X40N     11.85
XR        9.63
XY25      9.06
AR        9.00
OX40N     7.68
X200      1.84
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.97
X-MC    22.84
X-LC    19.95
M-SC    12.05
X-SC     8.26
H-MC     4.70
H-LC     2.96
L-SC     1.42
M-MC     1.36
M-LC     1.20
L-LC     0.96
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-5.27,39.31
IT,13.69,-16.04,75.95
FINANCE,9.42,-15.59,64.44
MISC,7.43,-23.20,73.67
PAINTS,5.97,-8.59,24.73
ELECTRICAL,5.91,-11.61,52.49
INSURANCE,4.47,0.49,34.58
PHARMA,3.89,-0.90,33.20
AUTO,3.35,-20.85,72.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3056479.0,21
XR,1293373.0,13
AR,1291848.0,10
X40,1023963.0,14
X40N,761924.0,9
OX40N,698888.0,10
XY25,352880.0,6
SR,279260.0,2
MH,73319.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3568274.0,25
M-SC,1344841.0,15
X-MC,1234018.0,16
X-LC,926131.0,11
X-SC,745009.0,8
H-MC,408980.0,3
L-SC,260205.0,3
H-LC,136057.0,2
M-LC,121356.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216559.0      6
           AR         876747.0      5
           XR         795995.0      7
M-SC       XY24       786708.0      6
X-MC       X40        445463.0      7
X-LC       X40        403982.0      5
X-MC       XY24       354050.0      3
H-SC       OX40N      326394.0      4
X-SC       X40N       296752.0      3
M-SC       OX40N      285016.0      5
H-SC       SR         279260.0      2
X-SC       XY24       273739.0      3
X-MC       X40N       252893.0      4
X-LC       XY24       232616.0      2
H-MC       AR         216173.0      2
X-LC       X40N       212279.0      2
H-MC       XY24       192807.0      1
X-MC       XY25       181612.0      2
X-SC       X40        174518.0      2
L-SC       XR         172727.0      2
M-SC       XR         148599.0      2
           AR         124518.0      2
M-LC       XR         121356.0      1
L-SC       OX40N       87478.0      1
X-LC       XY25        77254.0      2
H-LC       AR          74410.0      1
H-SC       MH          73319.0      1
H-LC       X200        61647.0      1
L-MC       XR          54696.0      1
M-MC       XY25        51585.0      1
L-LC       XY25        42429.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
